In [1]:
import osmnx as ox
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.lines import Line2D
from geopy import geocoders
import matplotlib.pyplot as plt
import re


In [24]:
def geocode_poi(poi):
    """
    Geocode a point of interest from the Photon geocoder, built on top of OpenStreetMap.

    Parameters:
    poi (str): The point of interest to geocode.
    
    Returns:
    tuple: The latitude and longitude of the point of interest.
    """

    geo_obj = geocoders.Photon()
    cleaned_poi = re.sub(r'[^\w\s]','',poi).replace(' ', '+').lower()
    result = geo_obj.geocode(cleaned_poi, exactly_one=True)
    return (result.latitude, result.longitude)


distance = 3000
cities = [
    'New York, NY', 
    'Los Angeles, CA', 
    'Chicago, IL', 
    'Houston, TX',
    'Phoenix, AZ',
    'Philadelphia, PA',
    'San Antonio, TX',
    'San Diego, CA',
    'Dallas, TX',
    'Austin, TX',
    'Jacksonville, FL',
    'San Jose, CA',
    'Fort Worth, TX',
    'Columbus, OH',
    'Charlotte, NC',
    'Indianapolis, IN',
    'San Francisco, CA',
    'Seattle, WA',
    'Denver, CO',
    'Oklahoma City, OK',
    'Nashville, TN',
    'El Paso, TX',
    'Washington, D.C.', 
    'Las Vegas, NV',
    'Boston, MA',
    'Portland, OR',
    'Louisville, KY',
    'Memphis, TN',
    'Detroit, MI',
    'Baltimore, MD',
    'Milwaukee, WI',
    'Albuquerque, NM',
    'Tucson, AZ',
    'Fresno, CA',
    'Sacramento, CA',
    'Mesa, AZ',
    'Kansas City, MO',
    'Atlanta, GA',
    'Colorado Springs, CO',
    'Omaha, NE',
    'Raleigh, NC',
    'Virginia Beach, VA',
    'Long Beach, CA',
    'Miami, FL',
    'Oakland, CA',
    'Minneapolis, MN',
    'Tulsa, OK',
    'Bakersfield, CA',
    'Tampa, FL',
    'Wichita, KS'
]

city_street_dict = {}
for city in cities:
    street_names = set()
    latitude, longitude = geocode_poi(city)
    G = ox.graph_from_point((latitude, longitude), distance, network_type="all", retain_all=True, simplify=False)
    H = ox.get_undirected(G)
    for _, edge in ox.graph_to_gdfs(H, nodes=False).fillna('').iterrows():
        text = edge['name']
        if text != '':
            street_names.add(text.split()[0])
    city_street_dict[city] = street_names

In [ ]:
inverted_dict = {}
for city, streets in city_street_dict.items():
    for street in streets:
        inverted_dict.setdefault(street, []).append(city)

sorted_dict = sorted(inverted_dict.items(), key=lambda item: len(item[1]), reverse=True)
[(x[0], len(x[1])) for x in sorted_dict][:25]

[('Market Street', 14),
 ('Main Street', 14),
 ('Elm Street', 14),
 ('Broadway', 13),
 ('State Street', 13),
 ('Spruce Street', 13),
 ('East 21st Street', 13),
 ('Jackson Street', 13),
 ('Oak Street', 13),
 ('Vine Street', 13),
 ('South 4th Street', 12),
 ('Park Avenue', 12),
 ('East 11th Street', 12),
 ('Pearl Street', 12),
 ('East 16th Street', 12),
 ('East 8th Street', 12),
 ('Franklin Street', 12),
 ('East 17th Street', 12),
 ('Jefferson Street', 12),
 ('East 9th Street', 12),
 ('Pine Street', 12),
 ('South 8th Street', 12),
 ('West 14th Street', 12),
 ('Union Street', 12),
 ('North 7th Street', 11)]